In [1]:
import unittest
from blinker import signal, Signal
import silq
from silq.pulses import Pulse, PulseMatch

In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
from qcodes.config.config import DotDict
class PulseConfig(DotDict):
    def __init__(self, name):
        self.__dict__['name'] = name
        self.__dict__['signal'] = signal('pulse_config:'+self.name)
        
    def __setitem__(self, key, val):
        super().__setitem__(key, val)
        self.signal.send(self, **{key: val})
        
    __setattr__ = __setitem__

In [4]:
class TestPulseSignals(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse')
        self.is_set = False 
        self.dict = {}

    def test_set_val(self):
        self.assertFalse(self.is_set)
        self.set_val(self)
        self.assertTrue(self.is_set)

    def test_signal_emit(self):
        p = Pulse()
        self.signal.connect(self.set_dict)
        self.assertFalse('t_start' in self.dict)
        p.t_start = 1
        self.assertEqual(1, self.dict['t_start'])

    def test_subsequent_pulses(self):
        p1 = Pulse(t_start=0, t_stop=10)
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 10)
        self.assertEqual(p1.t_stop, 10)
        
        p2 = Pulse(t_start=PulseMatch(p1, 't_stop', delay=1), duration=4)
        self.assertEqual(p2.t_start, 11)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 15)
        
        p1.t_stop = 14
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 14)
        self.assertEqual(p1.t_stop, 14)
        
        self.assertEqual(p2.t_start, 15)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 19)
        
        p2.t_start = 0
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)
        
        p1.t_stop = 20
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)

    def set_val(self, sender, **kwargs):
        self.is_set = True
        
    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [5]:
class TestPulseConfig(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse_config:read')
        self.pulse_config = PulseConfig('read')
        self.pulses_config = {'read': self.pulse_config}
        self.dict = {}

    def testSetItem(self):
        with self.assertRaises(KeyError):
            self.pulse_config['duration']
        self.pulse_config['duration'] = 1
        self.assertEqual(self.pulse_config['duration'], 1)
        self.assertEqual(self.pulse_config.duration, 1)

    def testSignal(self):
        self.signal.connect(self.set_dict)
        self.pulse_config.duration = 1
        self.assertIn('duration', self.pulse_config)
        self.assertEqual(self.pulse_config.get('duration', None), 1)

    def testPulseFromConfig(self):
        Pulse.pulses_config = self.pulses_config
        self.pulse_config.t_start = 10
        p = Pulse(name='read')
        self.assertEqual(p.t_start, 10)
        
        p.t_start = 20
        self.assertEqual(p.t_start, 20)
        
        self.pulse_config.t_start = 0
        self.assertEqual(p.t_start, 0)

    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [6]:
# Run tests    
suite_pulse_signals = unittest.TestLoader().loadTestsFromModule(TestPulseSignals())
suite_pulse_config = unittest.TestLoader().loadTestsFromModule(TestPulseConfig())
suite = unittest.TestSuite([suite_pulse_signals, suite_pulse_config])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_set_val (__main__.TestPulseSignals) ... ok
test_signal_emit (__main__.TestPulseSignals) ... ok
test_subsequent_pulses (__main__.TestPulseSignals) ... ok
testPulseFromConfig (__main__.TestPulseConfig) ... ok
testSetItem (__main__.TestPulseConfig) ... ok
testSignal (__main__.TestPulseConfig) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>